In [39]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer

# tr_offenseval_tweets_df = pd.read_csv('offenseval2020-turkish/offenseval-tr-testset-v1/offenseval-tr-testset-v1.tsv', sep="\t")
# tr_offenseval_labels_df = pd.read_csv('offenseval2020-turkish/offenseval-tr-testset-v1/offenseval-tr-labela-v1.tsv', sep=",")
HSO = pd.read_csv('hatespeech_offensive.csv', sep=',')

# OLID = pd.read_csv('OLIDv1.0/olid-training-v1.0.tsv', sep="\t")

# Trainer (has labels)
# trainer_tweets_df = pd.read_csv('extended_test_SOLID/test_a_tweets_all.tsv', sep="\t")
# trainer_labels_df = pd.read_csv('extended_test_SOLID/test_a_labels_all.csv', sep=",")

# Unsupervised learner (no labels)
# learner_tweets_df = pd.read_csv('SOLID_tweets_task_a_distant.tsv', sep="\t")

In [40]:
HSO.head()

,id,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [41]:

def remap_labels(df, col_name):
    df[col_name] = df[col_name].map({'OFF': 1, 'NOT': 0})
    return df
def remap_HSO_labels(df):
    df['class'] = df['class'].map({2: 0, 1: 1, 0: 1}) 
    return df

def remap_tweets(df, col_name):
    df.loc[:, col_name] = df.loc[:, col_name].str.replace('@USER', '') #Remove mentions (@USER)
    df.loc[:, col_name] = df.loc[:, col_name].str.replace('URL', '') #Remove URLs
    pat = r'(http|ftp|https):\/\/([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:\/~+#-]*[\w@?^=%&\/~+#-])' # remove URLS
    df.loc[:, col_name] = df.loc[:, col_name].str.replace(pat, ' ', regex=True)
    df.loc[:, col_name] = df.loc[:, col_name].str.replace('&amp', 'and') #Replace ampersand (&) with and
    df.loc[:, col_name] = df.loc[:, col_name].str.replace('&lt','') #Remove &lt
    df.loc[:, col_name] = df.loc[:, col_name].str.replace('&gt','') #Remove &gt
    df.loc[:, col_name] = df.loc[:, col_name].str.replace('\n',' ') #Remove \n
    df.loc[:, col_name] = df.loc[:, col_name].str.replace('\d+','', regex=True) #Remove numbers
    pat = r'[^\w\s]' # remove the emojis and other special characters
    df.loc[:, col_name] = df.loc[:, col_name].str.replace(pat, '', regex=True)
    #remove white space before and after the tweet
    df.loc[:, col_name] = df.loc[:, col_name].str.strip()
    df.loc[:, col_name] = df.loc[:, col_name].str.lower() #Lowercase
    
    # not needed outside of english
    contractions_dict = {
        r"\bive\b": "i have",
        r"\byoure\b": "you are",
        r"\bitd\b": "it would",
        r"\bcant\b": "cannot",
        r"\bdont\b": "do not",
        r"\bwont\b": "will not",
        r"\btho\b": "though",
        r"\bim\b": "i am",
        r"\bhes\b": "he is",
        r"\bshes\b": "she is",
        r"\bits\b": "it is",
        r"\bwere\b": "we are",
        r"\btheyre\b": "they are",
        r"\bshouldnt\b": "should not",
        r"\bwouldnt\b": "would not",
        r"\bcouldnt\b": "could not",
        r"\bisnt\b": "is not",
        r"\barent\b": "are not",
        r"\bhavent\b": "have not",
        r"\bhasnt\b": "has not",
        r"\bhadnt\b": "had not",
        r"\bwasnt\b": "was not",
        r"\bwerent\b": "were not"
    }
    
    # df[col_name] = df[col_name].replace(contractions_dict, regex=True) # Replace some popular contractions
    
    
    return df

In [42]:
# trainer_labels_df = remap_labels(trainer_labels_df, 'label')
# trainer_tweets_df = remap_tweets(trainer_tweets_df, 'tweet')
# learner_tweets_df = remap_tweets(learner_tweets_df, 'text')
# OLID = remap_labels(OLID, 'subtask_a')
# OLID = remap_tweets(OLID, 'tweet')
# tr_offenseval_labels_df = remap_labels(tr_offenseval_labels_df, 'label')
# tr_offenseval_tweets_df = remap_tweets(tr_offenseval_tweets_df, 'tweet')

HSO_df = remap_HSO_labels(HSO)
HSO_df = remap_tweets(HSO, 'tweet')


In [43]:
# OLID.rename(columns={'subtask_a': 'label'}, inplace=True)
# tr_offenseval_df.rename(columns={'subtask_a': 'label'}, inplace=True)
HSO_df.rename(columns={'class': 'label'}, inplace=True)


In [44]:
# OLID.to_csv('datasets/cleaned_OLID.tsv', sep='\t', index=False)
# trainer_df = trainer_tweets_df.merge(trainer_labels_df, on='id')
# trainer_df = trainer_tweets_df.merge(trainer_labels_df, on='id')
# tr_offenseval_test_df = tr_offenseval_tweets_df.merge(tr_offenseval_labels_df, on='id')
HSO_df = HSO_df.drop(columns=['count','hate_speech','offensive_language','neither'])

# trainer_df.to_csv('datasets/cleaned_SOLIDtest6K_trainer.tsv', sep='\t', index=False)
# learner_tweets_df.to_csv('datasets/cleaned_SOLID9M_learner.tsv', sep='\t',index=False)
HSO_df.to_csv('datasets/cleaned_hatespeech_offensive.tsv', sep='\t',index=False)



In [45]:
HSO_df.head(12)

,id,label,tweet
0,0,0,rt mayasolovely as a woman you shouldnt compla...
1,1,1,rt mleew boy dats coldtyga dwn bad for cuffin ...
2,2,1,rt urkindofbrand dawg rt sbabylife you ever fu...
3,3,1,rt c_g_anderson viva_based she look like a tranny
4,4,1,rt shenikaroberts the shit you hear about me m...
5,5,1,t_madison_x the shit just blows meclaim you so...
6,6,1,__brighterdays i can not just sit up and hate ...
7,7,1,selfiequeenbri cause im tired of you big bitch...
8,8,1,and you might not get ya bitch back and thats ...
9,9,1,rhythmixx_ hobbies include fighting mariam bitch
